In [ ]:
import numpy as np
import cv2 as cv
import glob

# INISIALISASI UKURAN CHECKERBOARD
# chessboardSize = (6,8)
chessboardSize = (7,10)
# chessboardSize = (7,7)
# frameSize = (1280,720)
frameSize = (640,480)
# world_scaling = 1.4
world_scaling = 2.5
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Pembuatan objek point seperti (0,0,0), (1,1,1)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)
objp = world_scaling* objp

# Pembuatan array untuk menyimpan titik objeck 2D dan 3D 
objpointsL = []  # Point 3D
objpointsR = []  # 
imgpointsL = [] # Point 2D
imgpointsR = [] 

# Pengambilan gambar checkerboard
imagesLeft = glob.glob(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\TES_KALIBRASI_KIRI\*.png')
imagesRight = glob.glob(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\TES_KALIBRASI_KANAN\*.png')
# imagesLeft = glob.glob(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\FINAL_KIRI\*.png')
# imagesRight = glob.glob(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\FINAL_KANAN\*.png')

for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    # Pengecekan gambar checkerboard serta pengubahan gambar menjadi grayscale 
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Pencarian titik ujung pada chessboard
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # Jika titik ujung ditemukan, titik objek dan gambar ditambahkan pada array 
    if retL and retR == True:

        objpointsL.append(objp)  
        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        objpointsR.append(objp)  
        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Gambar dengan titik ujung dari checkerboard terdeteksi ditampilkan
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imshow('img right', imgR)
        cv.waitKey(1000)
        print("chessboard found")
    elif retL == True:
        print("chessboard right not found")
    elif retR == True:
        print("chessboard left not found")
    else:
        print("chessboard not found")

cv.destroyAllWindows()


In [ ]:
# Pembuatan Kamera Matriks Lama dan Kamera Matriks Baru

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpointsL, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpointsR, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))

In [ ]:
print("Camera matrix L : ")
print(cameraMatrixL)
print("Camera matrix R : ")
print(cameraMatrixR)
print("New Camera matrix L : ")
print(newCameraMatrixL)
print("New Camera matrix R : ")
print(newCameraMatrixR)

In [ ]:
# Proses penghapusan dan pengecekan hasil penghapusan distorsi

cap = cv.VideoCapture(2)
cap2 = cv.VideoCapture(0)

cap2.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap2.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

while cap.isOpened():
    succes1, frameL = cap.read()
    succes2, frameR = cap2.read()

    # Proses penghapusan distorsi dilakukan
    frame_undistortedL = cv.undistort(frameL, cameraMatrixL, distL, None, newCameraMatrixL)
    frame_undistortedR = cv.undistort(frameR, cameraMatrixR, distR, None, newCameraMatrixR)

    # Proses penghapusan area hitam pada gambar dari hasil penghapusan distorsi
    xL, yL, wL, hL = roi_L
    xR, yR, wR, hR = roi_R

    croppedL = frame_undistortedL[yL:yL+hL, xL:xL+wL]
    croppedR = frame_undistortedR[yR:yR+hR, xR:xR+wR]

    cv.imshow("Left Undistorted", croppedL)
    cv.imshow("Right Undistorted", croppedR)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cap2.release()
cv.destroyAllWindows()

In [ ]:
# Hasil kamera matriks beserta nilai distorsi dan ROI disimpan untuk digunakan pada program pengambilan koordinat dari kamera
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\old_camera_matrix_L.npy', cameraMatrixL)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\distL.npy', distL)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\new_camera_matrix_L.npy', newCameraMatrixL)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_L\roi_L.npy', roi_L)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\old_camera_matrix_R.npy', cameraMatrixR)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\distR.npy', distR)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\new_camera_matrix_R.npy', newCameraMatrixR)
np.save(r'E:\Temp\SKRIPSI\tugas_rumusan_masalah\Camera Calibration\New_Camera_Parameter_R\roi_R.npy', roi_R)